<a href="https://colab.research.google.com/github/godsesaurab/data-science-projects/blob/main/Issues%26Incidents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Issues & Incidents Assignment

## Problem Statement :

two files: Issues and Incidents.

Each Incident represents the pain point extracted from a single customer support ticket. Issue is a pattern identified from multiple incidents. The provided incident file has links to issue files via issue_id.

The assignment is to write a prompt that will use both these files and for each issue propose a potential resolution.

In [1]:
import pandas as pd


In [2]:
json_file_path = '/content/spothopper_incidents.json'
df_incidents = pd.read_json(json_file_path)

In [3]:
json_file_path = '/content/spothopper_top5_issues.json'
df_issues = pd.read_json(json_file_path)

In [4]:
df_incidents.head()

,id,issueId,title,fullDescription
0,94365,5006,Request for integration of drink photos and op...,{'generalDescription': 'The customer requests ...
1,94286,5006,Request for menu update,{'generalDescription': 'The customer is reques...
2,94285,5006,Request for website updates,{'generalDescription': 'The customer is seekin...
3,94278,5006,Request for homepage specials button text change,{'generalDescription': 'The customer is reques...
4,94236,5006,Request for self-service landing page for week...,{'generalDescription': 'The customer requested...


In [5]:
df_issues.head()

,id,title,fullDescription
0,4965,Request for menu updates,{'generalDescription': 'Customers request assi...
1,5006,Request for assistance with website content up...,{'generalDescription': 'Customers are seeking ...
2,5016,Request for guidance on using platform features,{'generalDescription': 'Customers request inst...
3,5575,Request for adding or modifying website featur...,{'generalDescription': 'Customers request assi...
4,8176,Request for removing or updating menu items on...,{'generalDescription': 'Customers request assi...


In [6]:
from google.colab import userdata
import os

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY1')

print("Google API key fetched from Colab Secrets and configured as environment variable.")

Google API key fetched from Colab Secrets and configured as environment variable.


### Merging the Incidents and Issues Data for passing into prompt template

In [7]:
df_merged = pd.merge(df_incidents, df_issues, left_on='issueId', right_on='id', how='left', suffixes=('_incident', '_issue'))


In [8]:
df_merged

,id_incident,issueId,title_incident,fullDescription_incident,id_issue,title_issue,fullDescription_issue
0,94365,5006,Request for integration of drink photos and op...,{'generalDescription': 'The customer requests ...,5006,Request for assistance with website content up...,{'generalDescription': 'Customers are seeking ...
1,94286,5006,Request for menu update,{'generalDescription': 'The customer is reques...,5006,Request for assistance with website content up...,{'generalDescription': 'Customers are seeking ...
2,94285,5006,Request for website updates,{'generalDescription': 'The customer is seekin...,5006,Request for assistance with website content up...,{'generalDescription': 'Customers are seeking ...
3,94278,5006,Request for homepage specials button text change,{'generalDescription': 'The customer is reques...,5006,Request for assistance with website content up...,{'generalDescription': 'Customers are seeking ...
4,94236,5006,Request for self-service landing page for week...,{'generalDescription': 'The customer requested...,5006,Request for assistance with website content up...,{'generalDescription': 'Customers are seeking ...
...,...,...,...,...,...,...,...
495,92224,8176,Request for updating To-Go menu on website,{'generalDescription': 'The customer requested...,8176,Request for removing or updating menu items on...,{'generalDescription': 'Customers request assi...
496,92198,8176,Request for menu update on website,{'generalDescription': 'The customer requested...,8176,Request for removing or updating menu items on...,{'generalDescription': 'Customers request assi...
497,92183,8176,Request for posting lunch specials to website,{'generalDescription': 'The customer requested...,8176,Request for removing or updating menu items on...,{'generalDescription': 'Customers request assi...
498,92168,8176,Request for menu item updates and price changes,{'generalDescription': 'The customer requested...,8176,Request for removing or updating menu items on...,{'generalDescription': 'Customers request assi...


### Iteratively generating prompt from a prompt template - populating with incident and issue texts

In [9]:
df_merged['incident_description'] = df_merged['fullDescription_incident'].apply(lambda x: x.get('generalDescription', ''))
df_merged['issue_description'] = df_merged['fullDescription_issue'].apply(lambda x: x.get('generalDescription', ''))


prompt_template = """Given the following customer incident and its related issue, please propose a concrete, concise, and actionable resolution. Ensure the resolution directly addresses the core problem identified by both the incident and the issue. Please do not include any conversational filler, just the resolution.\n\nIncident Title: {incident_title}\nIncident Description: {incident_description}\n\nRelated Issue Title: {issue_title}\nRelated Issue Description: {issue_description}\n\nProposed Resolution:"""

df_merged['generated_prompt'] = df_merged.apply(
    lambda row: prompt_template.format(
        incident_title=row['title_incident'],
        incident_description=row['incident_description'],
        issue_title=row['title_issue'],
        issue_description=row['issue_description']
    ),
    axis=1
)




In [10]:
df_merged.head()

,id_incident,issueId,title_incident,fullDescription_incident,id_issue,title_issue,fullDescription_issue,incident_description,issue_description,generated_prompt
0,94365,5006,Request for integration of drink photos and op...,{'generalDescription': 'The customer requests ...,5006,Request for assistance with website content up...,{'generalDescription': 'Customers are seeking ...,The customer requests that drink photos and a ...,Customers are seeking help with making changes...,Given the following customer incident and its ...
1,94286,5006,Request for menu update,{'generalDescription': 'The customer is reques...,5006,Request for assistance with website content up...,{'generalDescription': 'Customers are seeking ...,The customer is requesting assistance with upd...,Customers are seeking help with making changes...,Given the following customer incident and its ...
2,94285,5006,Request for website updates,{'generalDescription': 'The customer is seekin...,5006,Request for assistance with website content up...,{'generalDescription': 'Customers are seeking ...,The customer is seeking assistance with making...,Customers are seeking help with making changes...,Given the following customer incident and its ...
3,94278,5006,Request for homepage specials button text change,{'generalDescription': 'The customer is reques...,5006,Request for assistance with website content up...,{'generalDescription': 'Customers are seeking ...,The customer is requesting to change the text ...,Customers are seeking help with making changes...,Given the following customer incident and its ...
4,94236,5006,Request for self-service landing page for week...,{'generalDescription': 'The customer requested...,5006,Request for assistance with website content up...,{'generalDescription': 'Customers are seeking ...,The customer requested a new landing page (e.g...,Customers are seeking help with making changes...,Given the following customer incident and its ...


### Using Gemini Flash 2.5 for faster output

In [11]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-flash-latest")

print("LLM initialized with gemini-flash-latest")

LLM initialized with gemini-flash-latest


### Iteratively calling LLM for 10 incidents using the generated prompts

In [12]:
from langchain_core.messages import HumanMessage

llm_resolutions_10 = []

for index, row in df_merged.head(10).iterrows():
    print(index)
    prompt_message = HumanMessage(content=row['generated_prompt'])
    try:
        response = llm.invoke([prompt_message])
        llm_resolutions_10.append(response.content)
    except Exception as e:
        llm_resolutions_10.append(f"Error generating resolution: {e}")


df_merged['llm_proposed_resolution_10'] = None
df_merged.loc[df_merged.head(10).index, 'llm_proposed_resolution_10'] = llm_resolutions_10

print(f"Generated {len(llm_resolutions_10)} LLM resolutions for the first 10 incidents.")
print("Displaying the first 2 sample prompts and their LLM resolutions:")
for i in range(min(2, len(llm_resolutions_10))):
    print(f"\n--- Sample Prompt {i+1} ---")
    print(df_merged.loc[i, 'generated_prompt'])
    print(f"--- LLM Proposed Resolution {i+1} ---")
    print(df_merged.loc[i, 'llm_proposed_resolution_10'])

0
1
2
3
4
5
6
7
8
9
Generated 10 LLM resolutions for the first 10 incidents.
Displaying the first 2 sample prompts and their LLM resolutions:

--- Sample Prompt 1 ---
Given the following customer incident and its related issue, please propose a concrete, concise, and actionable resolution. Ensure the resolution directly addresses the core problem identified by both the incident and the issue. Please do not include any conversational filler, just the resolution.

Incident Title: Request for integration of drink photos and options into the menu
Incident Description: The customer requests that drink photos and a list of drink options be integrated into the website's menu to enhance the visitor experience.

Related Issue Title: Request for assistance with website content updates
Related Issue Description: Customers are seeking help with making changes or updates to their restaurant website content, such as adding or removing information, updating images, or modifying specific website secti

## Final Output displaying Prompt and its output (10 incidents)

In [13]:
print(f"Generated {len(llm_resolutions_10)} LLM resolutions for the first 10 incidents.")
print("Displaying the sample prompts and their LLM resolutions:")
for i in range( len(llm_resolutions_10)):
    print(f"\n--- Sample Prompt {i+1} ---")
    print(df_merged.loc[i, 'generated_prompt'])
    print(f"--- LLM Proposed Resolution {i+1} ---")
    print(df_merged.loc[i, 'llm_proposed_resolution_10'])
    print("\n---------------------------------------------------------------")

Generated 10 LLM resolutions for the first 10 incidents.
Displaying the sample prompts and their LLM resolutions:

--- Sample Prompt 1 ---
Given the following customer incident and its related issue, please propose a concrete, concise, and actionable resolution. Ensure the resolution directly addresses the core problem identified by both the incident and the issue. Please do not include any conversational filler, just the resolution.

Incident Title: Request for integration of drink photos and options into the menu
Incident Description: The customer requests that drink photos and a list of drink options be integrated into the website's menu to enhance the visitor experience.

Related Issue Title: Request for assistance with website content updates
Related Issue Description: Customers are seeking help with making changes or updates to their restaurant website content, such as adding or removing information, updating images, or modifying specific website sections.

Proposed Resolution:
-

In [14]:
df_merged.to_csv('FinalResults.csv',index=False)